# 2024 OIBC Challenge
## (Deep Learning version skeleton 01)

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import datetime

In [2]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_name())
  print(torch.__version__)
  print(torch.version.cuda)
  x = torch.randn(1).cuda()
  print(x)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

NVIDIA GeForce GTX 1650
2.5.0+cu118
11.8
tensor([0.0838], device='cuda:0')


device(type='cuda', index=0)

In [3]:
elec_market_status = pd.read_csv('../../data_files/제주전력시장_현황데이터.csv')
elec_market_status['ts'] = pd.to_datetime(elec_market_status['ts'], unit='s')
start_index = elec_market_status.index[elec_market_status['ts']=='2024-02-29 14:00:00'][0]
elec_market_status = elec_market_status[start_index:]

elec_market_status.head()

,ts,공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
17188,2024-02-29 14:00:00,1347000.0,839000.0,0.0,196591.0,210378.0,508000.0,306000.0
17189,2024-02-29 14:05:00,1352000.0,842000.0,0.0,199446.0,213221.0,509000.0,309000.0
17190,2024-02-29 14:10:00,1351000.0,839000.0,0.0,198012.0,212961.0,511000.0,309000.0
17191,2024-02-29 14:15:00,1358000.0,841000.0,0.0,205018.0,219922.0,517000.0,318000.0
17192,2024-02-29 14:20:00,1364000.0,837000.0,0.0,211577.0,226343.0,528000.0,327000.0


In [4]:
elec_day_ahead = pd.read_csv('../../data_files/제주전력시장_시장전기가격_하루전가격.csv')
elec_day_ahead['ts'] = pd.to_datetime(elec_day_ahead['ts'], unit='s')
elec_day_ahead.head()

,ts,하루전가격(원/kWh)
0,2024-02-29 15:00:00,107.39
1,2024-02-29 16:00:00,107.39
2,2024-02-29 17:00:00,95.30
3,2024-02-29 18:00:00,87.89
4,2024-02-29 19:00:00,0.00


In [5]:
for idx in range(0, 1): #len(elec_day_ahead)
  one_hour_ago = elec_day_ahead['ts'][idx].to_pydatetime() - datetime.timedelta(hours=1)
  print(one_hour_ago, elec_day_ahead['ts'][idx])
  targets = elec_market_status[(one_hour_ago <= elec_market_status['ts']) & (elec_day_ahead['ts'][idx].to_pydatetime() > elec_market_status['ts'])]
  targets = targets.drop('ts', axis=1)
  print(torch.from_numpy(targets.to_numpy()))

  print(elec_day_ahead.drop('ts', axis=1).to_numpy()[idx][0])

2024-02-29 14:00:00 2024-02-29 15:00:00
tensor([[1347000.,  839000.,       0.,  196591.,  210378.,  508000.,  306000.],
        [1352000.,  842000.,       0.,  199446.,  213221.,  509000.,  309000.],
        [1351000.,  839000.,       0.,  198012.,  212961.,  511000.,  309000.],
        [1358000.,  841000.,       0.,  205018.,  219922.,  517000.,  318000.],
        [1364000.,  837000.,       0.,  211577.,  226343.,  528000.,  327000.],
        [1364000.,  826000.,       0.,  210374.,  225914.,  537000.,  337000.],
        [1373000.,  829000.,       0.,  219935.,  235739.,  544000.,  345000.],
        [1370000.,  823000.,       0.,  218018.,  232482.,  548000.,  348000.],
        [1389000.,  827000.,       0.,  235611.,  250743.,  562000.,  360000.],
        [1390000.,  818000.,       0.,  234741.,  250744.,  572000.,  371000.],
        [1393000.,  815000.,       0.,  242293.,  256841.,  578000.,  380000.]],
       dtype=torch.float64)
107.39


In [6]:
class ElecDataset(Dataset):
  def __init__(self, x_data, y_data):
    self.x_data = x_data
    self.y_data = y_data

  def __getitem__(self, index):
    one_hour_ago = self.y_data['ts'][index].to_pydatetime() - datetime.timedelta(hours=1)
    targets = self.x_data[(one_hour_ago <= self.x_data['ts']) & (self.y_data['ts'][index].to_pydatetime() > self.x_data['ts'])]
    targets = targets.drop('ts', axis=1)
    targets = torch.from_numpy(targets.to_numpy())
    # MEMO - padding
    if len(targets) < 12:
      targets = torch.nn.functional.pad(targets, (0, 0, 0, 12-len(targets)))
      # print(targets)
      # print('\n')

    target_y = self.y_data.drop('ts', axis=1).to_numpy()[index][0]

    return targets, target_y

  def __len__(self):
    return len(self.y_data)

In [7]:
scaler = MinMaxScaler()

columns_to_scale = elec_market_status.columns[1:]
elec_market_status[columns_to_scale] = scaler.fit_transform(elec_market_status[columns_to_scale])

dataset = ElecDataset(elec_market_status, elec_day_ahead)

In [8]:
dataloader = DataLoader(dataset=dataset,
                        batch_size=16,
                        shuffle=True,
                        drop_last=False)

# for epoch in range(int(len(dataset)/16)+1):
#   print(f"epoch: {epoch}")
#   for batch in dataloader:
#     X, y = batch
#     print(X.size(), y)

In [9]:
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
    super(LSTMModel, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    
    # Define the LSTM layer
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dtype=torch.float64)
    
    # Define a fully connected layer to produce output of size 1
    self.fc = nn.Linear(hidden_size, output_size, dtype=torch.float64)
  
  def forward(self, x):
    # Set initial hidden and cell states to zeros
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float64).to(x.device)  # hidden state
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float64).to(x.device)  # cell state
    
    # Forward propagate LSTM
    out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
    
    # Get the last time step output (last hidden state)
    out = out[:, -1, :]  # (batch_size, hidden_size)
    
    # Pass the output through the fully connected layer
    out = self.fc(out)  # (batch_size, 1)
    
    return out

In [13]:
input_size = 7  # Number of input features (7)
hidden_size = 16  # Hidden state size, you can choose other values too
output_size = 1  # Single output
num_layers = 3  # Number of LSTM layers (you can adjust this)

learning_rate = 0.01
epoches= 10


model = LSTMModel(input_size, hidden_size, output_size, num_layers)


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Example input: batch of 16 sequences, each of length 12, with 7 features per timestep
# it = iter(dataloader)
# input_tensor = next(it)[0]
# print(input_tensor)

# # Forward pass
# output = model(input_tensor)
# print(output)  # Expected output: (16, 1)

In [14]:
def train(model, train_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)  # Move model to GPU/CPU
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        
        running_loss = 0.0  # To keep track of loss
        for inputs, targets in train_loader:
            # Move data to the same device as the model
            inputs, targets = inputs.to(device), targets.to(device)
            targets = targets.view(-1, 1)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()  # Clear the gradients
            loss.backward()        # Compute gradients
            optimizer.step()        # Update model parameters

            running_loss += loss.item()

        # Print the loss after each epoch
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    print("Training complete.")

In [15]:
train(model, dataloader, criterion, optimizer, epoches, device)

Epoch [1/10], Loss: 11937.7819
Epoch [2/10], Loss: 4990.0548
Epoch [3/10], Loss: 2616.2151
Epoch [4/10], Loss: 1999.1004
Epoch [5/10], Loss: 1902.9440
Epoch [6/10], Loss: 1894.0407
Epoch [7/10], Loss: 1893.2088
Epoch [8/10], Loss: 1893.5194
Epoch [9/10], Loss: 1931.2521
Epoch [10/10], Loss: 1893.2481
Training complete.
